Description:

Ce notebook est utile pour afficher les courbes de tendance de SoH par modèle (pour l'instant tesla) et on peut même aller plus loin avec les courbe de tendance de SoH pour les versions (ou tesla code).

Il y'a la possibilité d'ajouter les intervals de confiance de ces courbes. 

In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join oem o 
            on o.id = vm.oem_id
            join battery b 
            on b.id = vm.battery_id
            where o.id in ('5b89ebf7-8774-41ec-97f8-7dd4e8a2fd0f');"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
# on drop MTY13/MT336/MT337
dbeaver_filtered = dbeaver_df[~dbeaver_df['version'].isin(['MT336','MT337'])][['version', 'soh', 'odometer', 'model_name', 'type', 'vin']].copy().dropna()


In [ ]:
px.scatter(dbeaver_df[['version', 'soh', 'odometer']],  x='odometer', y='soh', color="version")

## Graphs des tendances 

faire avec itteration 

In [ ]:
# Fonction logarithmique pour la tendance
def log_function(x, a, b ):
    return 1 + a * np.log1p(x/b) 


In [ ]:
data[(data['odometer'] < 5000) & (data['soh'] > .85)].soh.mean()

In [ ]:
data = dbeaver_filtered

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import t
from scipy.signal import savgol_filter


In [ ]:
sort_idx = np.argsort(data['odometer'])


In [ ]:
x_sorted, 

In [ ]:
y_upper

In [ ]:
x_data = data['odometer'].values
y_data = data['soh'].values

# Tri des données (important pour un bon tracé)
sort_idx = np.argsort(x_data)
x_sorted = x_data[sort_idx]
y_sorted = y_data[sort_idx]

# Ajustement de la fonction sur les données
popt, pcov = curve_fit(log_function, x_sorted, y_sorted)

# Calcul de l'écart-type local
window_size = 50  # Ajuste selon la densité des données
local_std = np.array([
    np.std(y_sorted[max(0, i-window_size):min(len(y_sorted), i+window_size)])
    for i in range(len(y_sorted))
])


# Appliquer un lissage de Savitzky-Golay sur l'écart-type local

x = np.linspace(0, 1, len(local_std))

# Interpolation linéaire entre le premier et le dernier point
smooth = np.linspace(local_std[0], local_std[-1], len(local_std))

# Calcul des intervalles de confiance
y_fit = log_function(x_sorted, *popt)
y_lower = y_fit - 1 * smooth
y_upper = y_fit +  1 * smooth
print(y_lower.max(), y_upper.max())

def log_function_min(x, a, b):
    return y_lower.max() + a * np.log1p(x/b) 
def log_function_max(x, a, b):
    return y_upper.max() + a * np.log1p(x/b) 


coef_lower, _ = curve_fit(log_function_min, x_sorted, y_lower)
lower_bound = log_function_min(x_sorted, *coef_lower)

coef_upper, _ = curve_fit(log_function_max, x_sorted, y_upper)
upper_bound = log_function_max(x_sorted, *coef_upper)




print(f"la fonction pour la borne sup est: 1 + {coef_upper[0]} * np.log1p(x/1000)" )
print(f"la fonction pour la borne inf est: 1 + {coef_lower[0]} * np.log1p(x/1000)" )
print(f"la fonction pour la tendance est: 1 + {popt[0]} * np.log1p(x/1000)" )
# Plot
plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, s=10, label='Données')
plt.plot(x_sorted, log_function(x_sorted, *popt), 'r-', label='Fit')
#plt.plot(x_sorted, y_lower, 'r-', label='Fit')
#plt.plot(x_sorted, y_upper, 'r-', label='Fit')
plt.plot(x_sorted, upper_bound, 'y-')
plt.plot(x_sorted, lower_bound, 'y-')
#plt.fill_between(x_sorted, y_lower, y_upper, color='lightgray', label='Intervalle 95%')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
popt

In [ ]:
dbeaver_filtered

In [ ]:
def get_trendline(df):
    x_data = df['odometer'].values
    y_data = df['soh'].values

    # Tri des données (important pour un bon tracé)
    sort_idx = np.argsort(x_data)
    x_sorted = x_data[sort_idx]
    y_sorted = y_data[sort_idx]

    # Ajustement de la fonction sur les données
    popt, _ = curve_fit(log_function, x_sorted, y_sorted)

    # Calcul de l'écart-type local
    window_size = 50  # Ajuste selon la densité des données
    local_std = np.array([
        np.std(y_sorted[max(0, i-window_size):min(len(y_sorted), i+window_size)])
        for i in range(len(y_sorted))
    ])

    # Appliquer un lissage de Savitzky-Golay sur l'écart-type local

    x = np.linspace(0, 1, len(local_std))

    # Interpolation linéaire entre le premier et le dernier point
    smooth = np.linspace(local_std[0], local_std[-1], len(local_std))

    # Calcul des intervalles de confiance
    y_fit = log_function(x_sorted, *popt)
    y_lower = y_fit - 1 * smooth
    y_upper = y_fit +  1 * smooth

    coef_lower, _ = curve_fit(log_function, x_sorted, y_lower)
    coef_upper, _ = curve_fit(log_function, x_sorted, y_upper)

    trendline = {"trendline": f"1 + {popt[0]} * np.log1p(x/1000)"}
    trendline_max ={"trendline": f"1 + {coef_upper[0]} * np.log1p(x/1000)"}
    trendline_min ={"trendline": f"1 + {coef_lower[0]} * np.log1p(x/1000)"}
    
    return trendline, trendline_max, trendline_min
    

In [ ]:
for model in dbeaver_filtered.version.unique():
    df_model_temp = dbeaver_filtered[dbeaver_filtered['version']==model].sort_values('odometer').copy()
    print(model)
    nbr_vin = df_model_temp.groupby('vin', observed=True, as_index=False)['soh'].max().vin.dropna().nunique()
    print(nbr_vin)
    if nbr_vin > 100:
    
        popt, covariance = curve_fit(log_function, df_model_temp['odometer'], 
                            df_model_temp['soh'])
        
        x_data = df_model_temp['odometer'].values
        y_data = df_model_temp['soh'].values

        # Tri des données (important pour un bon tracé)
        sort_idx = np.argsort(x_data)
        x_sorted = x_data[sort_idx]
        y_sorted = y_data[sort_idx]

        # Ajustement de la fonction sur les données
        popt, pcov = curve_fit(log_function, x_sorted, y_sorted)

        # Calcul de l'écart-type local
        window_size = 50  # Ajuste selon la densité des données
        local_std = np.array([
            np.std(y_sorted[max(0, i-window_size):min(len(y_sorted), i+window_size)])
            for i in range(len(y_sorted))
        ])

        # Appliquer un lissage de Savitzky-Golay sur l'écart-type local

        x = np.linspace(0, 1, len(local_std))

        # Interpolation linéaire entre le premier et le dernier point
        smooth = np.linspace(local_std[0], local_std[-1], len(local_std))

        # Calcul des intervalles de confiance
        y_fit = log_function(x_sorted, *popt)
        y_lower = y_fit - 1 * smooth
        y_upper = y_fit +  1 * smooth

        coef_lower, _ = curve_fit(log_function, x_sorted, y_lower)
        lower_bound = log_function(x_sorted, coef_lower)

        coef_upper, _ = curve_fit(log_function, x_sorted, y_upper)
        upper_bound = log_function(x_sorted, coef_upper)
        
        trendline = {"trendline": f"1 + {popt[0]} * np.log1p(x/1000)"}
        trendline_max ={"trendline": f"1 + {coef_upper[0]} * np.log1p(x/1000)"}
        trendline_min ={"trendline": f"1 + {coef_lower[0]} * np.log1p(x/1000)"}
        print(f"la fonction pour la borne sup est: 1 + { coef_upper[0]} * np.log1p(x/1000)" )
        print(f"la fonction pour la borne inf est: 1 + {coef_lower[0]} * np.log1p(x/1000)" )
        print(f"la fonction pour la tendance est: 1 + {popt[0]} * np.log1p(x/1000)" )   

In [ ]:

fig_model = go.Figure()
model_colors = {model: px.colors.qualitative.Plotly[i] for i, model in enumerate(dbeaver_filtered.model_name.unique())}

for model in dbeaver_filtered.model_name.unique():
    
    df_model_temp = dbeaver_filtered[dbeaver_filtered['model_name']==model].sort_values('odometer').copy()
    popt, covariance = curve_fit(log_function, df_model_temp['odometer'], 
                        df_model_temp['soh'])
    # x_vals = np.linspace(0.1,  250000, 500)
    # y_vals = log_function(x_vals, *popt)
    # perr = np.sqrt(np.diag(covariance))
    # delta = 1.96 * np.sqrt((perr[0] * np.log(df_model_temp['soh']))**2)  # Intervalle à 95%

    a_hat = popt[0]
    a_err = np.sqrt(covariance[0, 0]) 
    x_vals = np.linspace(0.1,  250000, 500)
    y_vals = log_function(x_vals, *popt)
    perr = np.sqrt(np.diag(covariance))
    delta = 1.96 * np.sqrt((perr[0] * np.log(x_vals))**2)  # Intervalle à 95%
    residuals = df_model_temp['soh'] - log_function(df_model_temp['odometer'], a_hat)
    sigma_res = np.std(residuals)
    log_term = np.log1p(x_vals / 1000)
    y_err_pred = 1.96 * np.sqrt( sigma_res**2)



    color = model_colors[model]  # Couleur unique pour le modèle
    color_rgba = color.replace('rgb', 'rgba').replace(')', ', 0.2)')  

    # Génération des valeurs ajustées
    fig_model.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{model} trend', line=dict(color=color)))
### Décommenter en dessous si on veut les intervals de confiance associé.      
    
    fig_model.add_trace(go.Scatter(
    x=np.concatenate([x_vals, x_vals[::-1]]), 
    y=np.concatenate([y_vals - y_err_pred, (y_vals + y_err_pred)[::-1]]),

    fill='toself',
    fillcolor=color_rgba,  
    showlegend=True,
    name="Intervalle de confiance 95%",
    opacity=.5
))


### Décommenter en dessous si on veut les points de SoH 
    # fig_model.add_traces(go.Scatter(x=df_model_temp['odometer'].values, y=df_model_temp['soh'], name=f'{model} value', mode='markers'))
    
    fig_model.update_xaxes(title='odometer')
    fig_model.update_yaxes(title='soh')
    fig_model.update_layout(title="Courbe de tendance pour les modèles tesla")

In [ ]:
# définition des différentes figures 
fig_version_3 = go.Figure()
fig_version_X = go.Figure()
fig_version_S = go.Figure()
fig_version_Y = go.Figure()

# courbe pour les tesla_code des model 3 
for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model 3"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:
        popt, covariance = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,  200000, 500)
        y_vals = log_function(x_vals, *popt)
        perr = np.sqrt(np.diag(covariance))
        delta = 1.96 * np.sqrt((perr[0] * np.log(x_vals))**2)  # Intervalle à 95%

        fig_version_3.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_3.update_xaxes(title='odometer')
        fig_version_3.update_yaxes(title='soh')
        fig_version_3.update_layout(title="Courbe de tendance pour les tesla code model 3")
        fig_version_3.add_trace(go.Scatter(
        x=np.concatenate([x_vals, x_vals[::-1]]), 
        y=np.concatenate([y_vals - delta, (y_vals + delta)[::-1]]),
        fill='toself',
        fillcolor=color_rgba,  
        showlegend=True,
        name="Intervalle de confiance 95%",
        opacity=.5
    ))


        
# courbe pour les tesla_code des model x 
for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model x"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:
        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,  200000, 500)
        y_vals = log_function(x_vals, *popt)

        fig_version_X.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_X.update_xaxes(title='odometer')
        fig_version_X.update_yaxes(title='soh')
        fig_version_X.update_layout(title="Courbe de tendance pour les tesla code model X")
            
# courbe pour les tesla_code des model y

for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model y"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:
        print(version)
        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,200000, 500)
        y_vals = log_function(x_vals, *popt)

        fig_version_Y.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_Y.update_xaxes(title='odometer')
        fig_version_Y.update_yaxes(title='soh')
        fig_version_Y.update_layout(title="Courbe de tendance pour les tesla code model Y")
        

# courbe pour les tesla_code des model s

for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model s"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:

        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,200000, 500)
        y_vals = log_function(x_vals, *popt)

        fig_version_S.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_S.update_xaxes(title='odometer')
        fig_version_S.update_yaxes(title='soh')
        fig_version_S.update_layout(title="Courbe de tendance pour les tesla code model S")

## Graphs

In [ ]:
fig_model.show()

In [ ]:
fig_version_X.show()

In [ ]:
fig_version_S.show()

In [ ]:
fig_version_3.show()

In [ ]:
fig_version_Y.show()

## répartition véhicules

In [ ]:
dbeaver_filtered.vin.nunique()

In [ ]:

dbeaver_filtered.groupby('vin')[["model_name"]].first()["model_name"].value_counts()

In [ ]:
dbeaver_filtered.groupby('vin')[["model_name", 'type']].first().groupby('model_name')['type'].value_counts()